# SQL Data Analysis Pipeline Project 
**Contributors: Tripti Agarwal, Seong Hee Park**

**DATA AND PROJECT DESCRIPTION:**

We are using an API that gets data from the Johns Hopkins Center for Systems Science and Engineering, about Covid-19 global case statistics. The API extracts data forall countries from the beginning of 2022 until date.

In [ ]:
import requests
import pandas as pd

In [ ]:
# https://rapidapi.com/axisbits-axisbits-default/api/covid-19-statistics

your_key = "###"

url = "https://covid-19-statistics.p.rapidapi.com/reports"
headers = {
	"X-RapidAPI-Key": your_key,
	"X-RapidAPI-Host": "covid-19-statistics.p.rapidapi.com"
}

start = '2022-01-01'
end = '2022-11-22'

date_range = pd.date_range(start, end, freq='d').strftime("%Y-%m-%d").tolist()

result = pd.DataFrame()

for d in date_range:
    querystring = {"date": d}
    print(d)
    response = requests.request("GET", url, headers=headers, params=querystring)

    # parse
    df = pd.DataFrame(response.json()['data'])[['date', 'confirmed', 'deaths', 'recovered','confirmed_diff','deaths_diff','recovered_diff','last_update','active','active_diff','fatality_rate']]
    
    # parse region
    df_reg = pd.DataFrame(columns=['iso','name','province','lat','long','cities'])
    for i in range(len(df)):
        df_temp = pd.DataFrame([response.json()['data'][i]['region']])
        df_reg = pd.concat([df_reg, df_temp], axis=0)
    df_reg = df_reg.reset_index()
    df_reg = df_reg.drop(columns=['index','cities'])

    df = pd.concat([df, df_reg],axis=1)
    
    result = pd.concat([result, df], axis=0)

result.to_csv('datacsv.csv')

In [2]:
#IMPORTING THE CSV WITH DATA EXTRACTED FROM API

data = pd.read_csv('datacsv.csv')
data.head()

,Unnamed: 0,date,confirmed,deaths,recovered,confirmed_diff,deaths_diff,recovered_diff,last_update,active,active_diff,fatality_rate,iso,name,province,lat,long
0,0,2022-01-01,158107,7356,0,23,0,0,2022-01-02 04:20:52,150751,23,0.0465,AFG,Afghanistan,NaN,33.9391,67.7100
1,1,2022-01-01,210224,3217,0,0,0,0,2022-01-02 04:20:52,207007,0,0.0153,ALB,Albania,NaN,41.1533,20.1683
2,2,2022-01-01,218818,6284,0,386,8,0,2022-01-02 04:20:52,212534,378,0.0287,DZA,Algeria,NaN,28.0339,1.6596
3,3,2022-01-01,23740,140,0,0,0,0,2022-01-02 04:20:52,23600,0,0.0059,AND,Andorra,NaN,42.5063,1.5218
4,4,2022-01-01,82398,1772,0,805,2,0,2022-01-02 04:20:52,80626,803,0.0215,AGO,Angola,NaN,-11.2027,17.8739


**UPLOADING DATA TO CLOUD**

In [3]:
import pymysql

In [4]:
endpoint= "sqlproject.ctewfhqakgee.us-west-1.rds.amazonaws.com"
username= "###"
password= "###" 

connection = pymysql.connections.Connection(host = endpoint, user= username, password= password)

**CREATING FUNCTIONS TO RUN BASIC SQL PROMPTS**

In [5]:

def create_db(dbname):
    connection.cursor().execute(f"CREATE DATABASE IF NOT EXISTS {dbname}")
    connection.commit()
    print("DB Created")

def create_table(dbname, tablename, formatted_sql_columns):
    connection.cursor().execute(f"USE {dbname}")
    connection.cursor().execute(f"CREATE TABLE IF NOT EXISTS {tablename} ({formatted_sql_columns})")
    #formatted_sql_columns is (Col1 datatype, Col2 datatype...)
    connection.commit()

def delete_table(dbname, tablename):
    connection.cursor().execute(f"USE {dbname}")
    connection.cursor().execute(f"DROP TABLE IF EXISTS {tablename}")
    connection.commit()

**DEFINING THE STRING FOR COLUMN NAMES AND ITS DATATYPES**

In [6]:
list(data.columns)

['Unnamed: 0',
 'date',
 'confirmed',
 'deaths',
 'recovered',
 'confirmed_diff',
 'deaths_diff',
 'recovered_diff',
 'last_update',
 'active',
 'active_diff',
 'fatality_rate',
 'iso',
 'name',
 'province',
 'lat',
 'long']

Here, the first column is reduntant. Further, there are certain column names with SQL native words: date, active, name, long.

We will change those column names and update the dataset.

In [7]:
#dropping 1st column of our data
data_final = data.iloc[:, 1:]

In [8]:
#changing column names
cols = list(data_final.columns)     #making a list of column names

In [9]:
#changing the particular column names
cols[0] = "observed_date"
cols[8] = "active_cases"
cols[12] = "country_name"
cols[-2] = "latitude"  #just for uniformity
cols[-1] = "longitude"

In [10]:
cols         #list of column names without native words

['observed_date',
 'confirmed',
 'deaths',
 'recovered',
 'confirmed_diff',
 'deaths_diff',
 'recovered_diff',
 'last_update',
 'active_cases',
 'active_diff',
 'fatality_rate',
 'iso',
 'country_name',
 'province',
 'latitude',
 'longitude']

In [11]:
#changing col names in the dataframe
data_final.columns = cols

In [12]:
data_final.head()

,observed_date,confirmed,deaths,recovered,confirmed_diff,deaths_diff,recovered_diff,last_update,active_cases,active_diff,fatality_rate,iso,country_name,province,latitude,longitude
0,2022-01-01,158107,7356,0,23,0,0,2022-01-02 04:20:52,150751,23,0.0465,AFG,Afghanistan,NaN,33.9391,67.7100
1,2022-01-01,210224,3217,0,0,0,0,2022-01-02 04:20:52,207007,0,0.0153,ALB,Albania,NaN,41.1533,20.1683
2,2022-01-01,218818,6284,0,386,8,0,2022-01-02 04:20:52,212534,378,0.0287,DZA,Algeria,NaN,28.0339,1.6596
3,2022-01-01,23740,140,0,0,0,0,2022-01-02 04:20:52,23600,0,0.0059,AND,Andorra,NaN,42.5063,1.5218
4,2022-01-01,82398,1772,0,805,2,0,2022-01-02 04:20:52,80626,803,0.0215,AGO,Angola,NaN,-11.2027,17.8739


**CREATING THE DATABASE AND TABLE ON MYSQL**

In [14]:
#setting up parameters to run the database and table creation functions
dbname = "Finals"
tablename = "covid"

In [13]:
#redefining the formatted string
formatted_sql_columns = 'observed_date DATE, confirmed INT, deaths INT, recovered INT, confirmed_diff INT, \
deaths_diff INT, recovered_diff INT, last_update DATETIME, active_cases INT, active_diff INT, fatality_rate INT, \
iso TEXT, country_name TEXT, province TEXT, latitude FLOAT, longitude FLOAT'

In [15]:
create_db(dbname)
delete_table(dbname, tablename)
create_table(dbname, tablename, formatted_sql_columns)

DB Created


**INSERTING DATA INTO TABLE**

In [17]:
#creating function
def add_row(table_name, dbname, columns_sql, row_data):
    esses= ("%s, "* len(columns_sql))[:-2]
    fixed_columns = ",".join(columns_sql)
    
    sql = f"INSERT INTO {table_name} ({fixed_columns}) VALUES ({esses})"
    
    connection.cursor().execute(f"USE {dbname}")
    connection.cursor().execute(sql, row_data)
    connection.commit()
    
    return({"Status : 200"})

In [49]:
# Slice data (3 months)

tdate = '2022-08-22'

data_final['observed_date'] = pd.to_datetime(data_final['observed_date'])
df_subset = data_final[data_final.observed_date >= tdate].reset_index().drop(columns='index')
df_subset.observed_date = df_subset.observed_date.astype('str')
df_subset

,observed_date,confirmed,deaths,recovered,confirmed_diff,deaths_diff,recovered_diff,last_update,active_cases,active_diff,fatality_rate,iso,country_name,province,latitude,longitude
0,2022-08-22,191040,7767,0,397,5,0,2022-08-23 04:20:56,183273,392,0.0407,AFG,Afghanistan,NaN,33.939100,67.710000
1,2022-08-22,326181,3576,0,104,0,0,2022-08-23 04:20:56,322605,104,0.0110,ALB,Albania,NaN,41.153300,20.168300
2,2022-08-22,269894,6878,0,89,0,0,2022-08-23 04:20:56,263016,89,0.0255,DZA,Algeria,NaN,28.033900,1.659600
3,2022-08-22,45975,154,0,0,0,0,2022-08-23 04:20:56,45821,0,0.0033,AND,Andorra,NaN,42.506300,1.521800
4,2022-08-22,102636,1917,0,0,0,0,2022-08-23 04:20:56,100719,0,0.0187,AGO,Angola,NaN,-11.202700,17.873900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73462,2022-11-22,11512138,43169,0,316,0,0,2022-11-23 04:21:06,11468969,316,0.0037,VNM,Vietnam,NaN,14.058300,108.277200
73463,2022-11-22,703036,5708,0,0,0,0,2022-11-23 04:21:06,697328,0,0.0081,PSE,West Bank and Gaza,NaN,31.952200,35.233200
73464,2022-11-22,11945,2159,0,0,0,0,2022-11-23 04:21:06,9786,0,0.1807,YEM,Yemen,NaN,15.552727,48.516388
73465,2022-11-22,333721,4019,0,36,2,0,2022-11-23 04:21:06,329702,34,0.0120,ZMB,Zambia,NaN,-13.133900,27.849300


In [ ]:
# adding data
for i, row in df_subset.iterrows():
    rowdata = [str(i) for i in list(row)]
    add_row(tablename, dbname, cols, rowdata)